In [9]:

import io
import dash
import numpy as np
import pandas as pd
import dash_html_components as html
import base64

from jupyter_dash import JupyterDash 
import dash_cytoscape as cyto
from dash.dependencies import Input, Output, State
import dash_core_components as dcc

from inverse_covariance import (
    QuicGraphicalLasso,
    QuicGraphicalLassoCV,
    QuicGraphicalLassoEBIC,
    AdaptiveGraphicalLasso,
    ModelAverage,
)


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Network Capstone Design'



app.layout = html.Div([
        dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    
   cyto.Cytoscape(
            id='cytoscape',
            style={'width': '100%', 'height': '550px'},
           	layout={'name': 'circle'},
            stylesheet=[
    {
        "selector": 'node',
        'style': {
            "label": "data(label)", 
            "background-color": "#07ABA0", 
            "color": "#008B80",
            "width": "6px",
            "height": "6px",
            "Label Font Size" : "1px"
        }
    },
    {
        "selector": 'edge', 
        "style": {
            "line-color": "#C5D3E2",
            'opacity': 'data(weight)',
            #'width': 'abs(data(weight))*10px'
    }
}]
        )
    ])


@app.callback(Output('cytoscape', 'elements'),
             [Input("upload-data", "contents"), Input("upload-data", "filename")])
    
def networkGraph(contents, filename):    
    if contents:
        contents= contents[0]
        filename= filename[0]

        content_type, content_string = contents.split(",")
        decoded = base64.b64decode(content_string)

        try:
            if "csv" in filename:
                # Assume that the user uploaded a CSV or TXT file
                df= pd.read_csv(io.StringIO(decoded.decode("utf-8")))
            elif "xls" in filename:
                # Assume that the user uploaded an excel file
                df= pd.read_excel(io.BytesIO(decoded))
            elif "txt" or "tsv" in filename:
                # Assume that the user upl, delimiter= r'\s+'oaded an excel file
                df= pd.read_csv(io.StringIO(decoded.decode("utf-8")), delimiter=r"\s+")
        except Exception as e:
            print(e)
            return html.Div(["There was an error processing this file."])
        
        #df = df.corr(method='spearman')
        df = df.dropna()
        estimator = QuicGraphicalLassoEBIC(init_method='spearman', gamma=0.6, lam=0.2)
        estimator.fit(df.values)
        
        df = pd.DataFrame(np.triu(estimator.covariance_, 1))
        df = df.stack().reset_index()
        
        df.columns = ['from', 'to', 'weight']
        nodes = set()

        cy_edges = []
        cy_nodes = []

        for index, row in df.iterrows():
            source, target, weight = row['from'], row['to'], row['weight']
            if(weight != 0):
                nodes.add(source)
                cy_nodes.append({"data": {"id": source, "label": source}})
                nodes.add(target)
                cy_nodes.append({"data": {"id": target, "label": target}})

                cy_edges.append({
                    'data': {
                        'source': source,
                        'target': target,
                        'weight': weight
                    }
                })
            
        elements = cy_edges+cy_nodes

        return elements
    
    else:
        return {}
    
    

app.run_server(mode='external')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning: The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.
  func()


Dash app running on http://127.0.0.1:8050/


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/linalg/linalg.py:2099: RuntimeWarning: overflow encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/linalg/linalg.py:2099: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
